In [1]:
test_fold=3 #choose fold from[0,1,2,3,4] 

In [2]:

!pip install openpyxl
!pip install xlrd==2.0.1
!pip install Pandas==1.3.5
# !pip install Pandas==1.1.5
!pip install  gdown
import tensorflow as tf

import numpy as np
import pandas as pd

def normalize1(X, means1=None, std1=None, means2=None, std2=None, feat_filt=None, norm='tanh_norm'):
    if std1 is None:
        std1 = np.nanstd(X, axis=0)
    if feat_filt is None:
        feat_filt = std1!=0
    X = X[:,feat_filt]
    X = np.ascontiguousarray(X)
    if means1 is None:
        means1 = np.mean(X, axis=0)
    X = (X-means1)/std1[feat_filt]
    if norm == 'norm':
        return(X, means1, std1, feat_filt)
    elif norm == 'tanh':
        return(np.tanh(X), means1, std1, feat_filt)
    elif norm == 'tanh_norm':
        X = np.tanh(X)
        if means2 is None:
            means2 = np.mean(X, axis=0)
        if std2 is None:
            std2 = np.std(X, axis=0)
        X = (X-means2)/std2
        X[:,std2==0]=0
        return(X, means1, std1, means2, std2, feat_filt) 


    
def get_data():
  !gdown https://drive.google.com/uc?id=1m69fQ5l7q3dwQeud5M1nUVNhmrTGFbZc
  smiles=pd.read_excel('pubchem.xls', header=None)
  smiles=np.array(smiles)


  !gdown --id 10ztxKtGSVU7p9yPoCsTnapVLThWpjjYT
  data_to_repeat=pd.read_excel('oneil.xlsx', header=None)
  data_to_repeat=np.array(data_to_repeat)
  

  !gdown https://drive.google.com/uc?id=1m69fQ5l7q3dwQeud5M1nUVNhmrTGFbZc
  unique_drugs=pd.read_excel('pubchem.xls', header=None)
  unique_drugs=np.array(unique_drugs)         

  !gdown --id 1rVt2qEH-LMzk86ig6c8Qjll-08LhsxBv
  feature_cell=pd.read_excel('cell_expression875.xlsx')
  feature_cell=np.array(feature_cell)
  feature_cell[:,1:]=(2**feature_cell[:,1:])-1
    
  !gdown --id 14upx46iIPcO80y_tcvbaS3uwMkZfKLVw
  redkit_drug=pd.read_excel('redkit_drug.xlsx',header=None)
  redkit_drug=np.array(redkit_drug)
  
    
#   !gdown --id 1Yf9YQRSq3Bf0bIwncpoANS5Ui1H3W_fH
#   graph=pd.read_excel('graph_emb1.xlsx',header=None)

#   !gdown 1uB-9BAHzw2OKrH4T1BEsYNSgj__RoYVr
#   graph=pd.read_excel('views0a0123.xlsx',header=None)  
    
  !gdown 1nkNG2Z38WQgM1Ap8Wrg5aVbjC-NNUO6S
  graph=pd.read_excel('views0123.xlsx',header=None)

  graph=np.array(graph)
  
    
  !gdown --id 1HQpJQ6yeZYnGK6Uy0AyIvzYsNkzBAeOD
  model = tf.keras.models.load_model("model.h5")

  !gdown --id 1AZX2h806qMcMp63hnJB81JFYMjzEChpA
  unique_drugs1=pd.read_excel('unique394_drugl.xlsx', header=None)
  unique_drugs1=np.array(unique_drugs1)


  return smiles[1:,1],data_to_repeat[1:,:],unique_drugs[1:,0],feature_cell,redkit_drug,model,graph,unique_drugs1



def repeat_smiles1(data_to_repeat,unique_drugs,feature,unique_cell,redkit_drug,graph):
  unique_redkit_feature=redkit_drug[:,1:]
  unique_redkit_name=redkit_drug[:,0]
  unique_feature=unique_cell[:,1:]
  unique_name=unique_cell[:,0]
  d1=data_to_repeat[:,0]
  d2=data_to_repeat[:,1]
  c=data_to_repeat[:,2]
  unique_drugs=feature[:,0]
  feature=feature[:,1:]  
  f_drug1=[]
  f_drug2=[]
  feature_cell=[]
  redkit_d1=[]
  redkit_d2=[]
  graph1=[]
  graph2=[]
  for i in range(len(d1)):
    n1=d1[i]
    n2=d2[i]
    cc=c[i]
    k1= [m for m, v in enumerate(unique_drugs) if n1 in v]
    k2=[m for m, v in enumerate(unique_drugs) if n2 in v]
    cc1=[m for m, v in enumerate(unique_name) if cc in v]
    r1= [m for m, v in enumerate(unique_redkit_name) if n1 in v]
    r2=[m for m, v in enumerate(unique_redkit_name) if n2 in v]
    f_drug1.append(feature[k1[0]])
    f_drug2.append(feature[k2[0]])
    graph1.append(graph[k1[0]])
    graph2.append(graph[k2[0]])
    feature_cell.append(unique_feature[cc1[0]])
    redkit_d1.append(unique_redkit_feature[r1[0]])
    redkit_d2.append(unique_redkit_feature[r2[0]])

  return f_drug1,f_drug2,feature_cell,redkit_d1,redkit_d2,graph1,graph2




def train_test_input(f_drug1,f_drug2,cell_line,index_train,index_test,synery,class1,redkit_d1,redkit_d2,graph1,graph2):
  train_f_drug1=[]
  train_f_drug2=[]
  train_cell_line=[]
  train_synergy=[]
  train_class=[]
  train_redkit_d1=[]
  train_redkit_d2=[]
  test_f_drug1=[]
  test_f_drug2=[]
  test_cell_line=[]
  test_synergy=[]
  test_class=[]
  test_redkit_d1=[]
  test_redkit_d2=[]
  train_graph1=[]
  test_graph1=[]
  train_graph2=[]
  test_graph2=[]
  for i in range(len(index_train)):
      
      train_f_drug1.append(f_drug1[index_train[i]])
      train_f_drug2.append(f_drug2[index_train[i]])
      train_cell_line.append(cell_line[index_train[i]])
      train_synergy.append(synergy[index_train[i]])
      train_class.append(class1[index_train[i]])
      train_redkit_d1.append(redkit_d1[index_train[i]])
      train_redkit_d2.append(redkit_d2[index_train[i]])
      train_graph1.append(graph1[index_train[i]])
      train_graph2.append(graph2[index_train[i]])

  for ii in range(len(index_test)):
      
      test_f_drug1.append(f_drug1[index_test[ii]])
      test_f_drug2.append(f_drug2[index_test[ii]])
      test_cell_line.append(cell_line[index_test[ii]])
      test_synergy.append(synergy[index_test[ii]])
      test_class.append(class1[index_test[ii]])
      test_redkit_d1.append(redkit_d1[index_test[ii]])
      test_redkit_d2.append(redkit_d2[index_test[ii]])
      test_graph1.append(graph1[index_test[ii]])
      test_graph2.append(graph2[index_test[ii]])

  return train_f_drug1,train_f_drug2,train_cell_line,test_f_drug1,test_f_drug2,test_cell_line,train_synergy,train_class,test_synergy,test_class,train_redkit_d1,train_redkit_d2,test_redkit_d1,test_redkit_d2,train_graph1,test_graph1,train_graph2,test_graph2



def preprocess(index_train,index_test):
    index_train1=[]
    index_test1=[]
    index_train2=(index_train)[0]
    index_test2=(index_test)[0]
    for i in range(len((index_train2))):
        index_train1.append((index_train2[i]))
        
    for ii in range(len(index_test2)):
        index_test1.append((index_test2[ii]))
        
    return index_train1,index_test1

def get_data_me2(s):
    
    

    !gdown 1C7Z2ziPdQVzH3omIdIfyJa7VUmog4IIk
    labels = pd.read_csv('oneil.csv', index_col=0) 
    
    h=len(np.array(labels))
    #labels are duplicated for the two different ways of ordering in the data
    labels = pd.concat([labels, labels]) 
    
    test_fold =s
   
    idx_train = np.where(labels['fold']!=test_fold)
    

    idx_test = np.where(labels['fold']==test_fold)
#     
#    
    return idx_train,idx_test





def convert_tobin(cc):
    cb=[]
    for i in range(len(cc)):
        if(cc[i]>=0.5):
            cb.append(1)
        else:
            cb.append(0)
    return cb


def norm1(train_cell_line,test_cell_line,norm="tanh_norm"):
# norm = "norm"
    if norm == "tanh_norm":
        train_cell_line, mean, std, mean2, std2, feat_filt = normalize1(train_cell_line, norm=norm)
        test_cell_line, mean, std, mean2, std2, feat_filt = normalize1(test_cell_line, mean, std, mean2, std2, 
                                                              feat_filt=feat_filt, norm=norm)
    else:
        train_cell_line, mean, std, feat_filt = normalize1(train_cell_line, norm=norm)
        test_cell_line, mean, std, feat_filt = normalize1(test_cell_line, mean, std, feat_filt=feat_filt, norm=norm)
    
    return train_cell_line,test_cell_line



    
     

smiles,data_to_repeat,unique_drugs,unique_cell,redkit_drug,ddi_model,graph,feature=get_data()

data_to_repeat=np.r_[data_to_repeat,data_to_repeat]
l=int((data_to_repeat.shape[0])/2)
data_to_repeat[l:,0]=data_to_repeat[0:l,1]
data_to_repeat[l:,1]=data_to_repeat[0:l,0]
synergy=data_to_repeat[:,3]

class1=[]
for i in range(len(synergy)):
 if(synergy[i]>=30):
    class1.append(1)
 elif(synergy[i]<0): 
    class1.append(0)
 else:
    class1.append(2)    


f_drug1,f_drug2,feature_cell,redkit_d1,redkit_d2,graph1,graph2=repeat_smiles1(data_to_repeat,unique_drugs,feature,unique_cell,redkit_drug,graph)




index_train,index_test=get_data_me2(test_fold)

index_train,index_test=preprocess(index_train,index_test)
train_f_drug1,train_f_drug2,train_cell_line,test_f_drug1,test_f_drug2,test_cell_line,train_synergy,train_class,test_synergy,test_class,train_redkit_d1,train_redkit_d2,test_redkit_d1,test_redkit_d2,train_graph1,test_graph1,train_graph2,test_graph2=train_test_input(f_drug1,f_drug2,feature_cell,index_train,index_test,synergy,class1,redkit_d1,redkit_d2,graph1,graph2)
train_cell_line=np.array(train_cell_line).astype(float)
test_cell_line=np.array(test_cell_line).astype(float)  
train_f_drug1=np.array(train_f_drug1).astype(float)
test_f_drug1=np.array(test_f_drug1).astype(float)
train_f_drug2=np.array(train_f_drug2).astype(float)
test_f_drug2=np.array(test_f_drug2).astype(float)

train_graph1=np.array(train_graph1).astype(float)
test_graph1=np.array(test_graph1).astype(float) 
train_graph2=np.array(train_graph2).astype(float)
test_graph2=np.array(test_graph2).astype(float)

train_cell_line,test_cell_line=norm1(train_cell_line,test_cell_line)

train_f_drug1,test_f_drug1=norm1(train_f_drug1,test_f_drug1)
train_f_drug2,test_f_drug2=norm1(train_f_drug2,test_f_drug2)

train_graph1,test_graph1=norm1(train_graph1,test_graph1)
train_graph2,test_graph2=norm1(train_graph2,test_graph2)




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 77.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: Pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.6.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
beatrix-jupyterlab 2023.621.222118 requires jupyter-server~=1.16, but you have jupyter-server 2.6.0 which is incompatible.
cudf 23.6.1 requires protobuf<4.22,>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
cuml 23.6.0 requires dask==2023.3.2, but you have dask 2023.7.0 which is inc

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Downloading...
From: https://drive.google.com/uc?id=1m69fQ5l7q3dwQeud5M1nUVNhmrTGFbZc
To: /kaggle/working/pubchem.xls
100%|██████████████████████████████████████| 27.1k/27.1k [00:00<00:00, 52.6MB/s]
/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=10ztxKtGSVU7p9yPoCsTnapVLThWpjjYT
To: /kaggle/working/oneil.xlsx
100%|█████████████████████████████████████████| 850k/850k [00:00<00:00, 124MB/s]
Downloading...
From: https://drive.google.com/uc?id=1m69fQ5l7q3dwQeud5M1nUVNhmrTGFbZc
To: /kaggle/working/pubchem.xls
100%|██████████████████████████████████████| 27.1k/27.1k [00:00<00:00, 56.3MB/s]
/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use 

In [3]:
!pip install tensorflow
!pip install spektral
from tensorflow import keras
from tensorflow.keras.layers import Minimum,Maximum,Add,Maximum,PReLU, Flatten,Reshape,Dropout, Input,Dense,Add,concatenate,BatchNormalization, Activation#,MultiHeadAttention,AdditiveAttention
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from spektral.layers import GATConv, GlobalAvgPool,GlobalMaxPool, GCNConv,GlobalAttentionPool
from spektral.transforms import LayerPreprocess
from sklearn.metrics import confusion_matrix, mean_squared_error,mean_absolute_error,r2_score#,AUC
from scipy.stats import pearsonr,spearmanr
from sklearn.metrics import roc_curve,auc,accuracy_score,precision_score,cohen_kappa_score,precision_recall_curve,average_precision_score,roc_auc_score
from keras.utils.vis_utils import plot_model
from tensorflow.keras import regularizers
np.random.seed(10)

train_cell_line=tf.convert_to_tensor(train_cell_line)
test_cell_line=tf.convert_to_tensor(test_cell_line)
train_synergy=tf.convert_to_tensor(train_synergy)
train_class=tf.convert_to_tensor(train_class)
test_synergy=tf.convert_to_tensor(test_synergy)
test_class=tf.convert_to_tensor(test_class)
test_redkit_d1=tf.convert_to_tensor(test_redkit_d1)
test_redkit_d2=tf.convert_to_tensor(test_redkit_d2)
train_f_drug1=tf.convert_to_tensor(train_f_drug1)
test_f_drug1=tf.convert_to_tensor(test_f_drug1)
train_f_drug2=tf.convert_to_tensor(train_f_drug2)
test_f_drug2=tf.convert_to_tensor(test_f_drug2)
train_graph1=tf.convert_to_tensor(train_graph1)
test_graph1=tf.convert_to_tensor(test_graph1)
train_graph2=tf.convert_to_tensor(train_graph2)
test_graph2=tf.convert_to_tensor(test_graph2)



def ddi_fun(train_redkit_d1,train_redkit_d2):
    train_redkit_d1=np.array(train_redkit_d1).astype(float)
    train_redkit_d2=np.array(train_redkit_d2).astype(float)
    ddi_extractor = keras.Model(
    inputs=ddi_model.inputs,
    outputs=ddi_model.get_layer(name="features").output,
    )

    train_ddi=ddi_extractor.predict([train_redkit_d1,train_redkit_d2])

    return train_ddi

test_ddi=ddi_fun(test_redkit_d1,test_redkit_d2)
train_ddi=ddi_fun(train_redkit_d1,train_redkit_d2)
train_ddi=np.array(train_ddi).astype(float)
test_ddi=np.array(test_ddi).astype(float)

train_ddi,test_ddi=norm1(train_ddi,test_ddi)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 7.0 MB/s eta 0:00:00
1139/1139 [==============================] - 3s 2ms/step


In [4]:

import numpy as np
import matplotlib.pyplot as plt

# import tensorflow_datasets as tfds
import tensorflow as tf
def scaled_dot_product_attention(q, k, v, mask=None):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits)#, axis=-1)#tf.nn.linear()#, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output,attention_weights 

#multi-head

class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model,activation='relu',use_bias='true')
    self.wk = tf.keras.layers.Dense(d_model,activation='relu',use_bias='true')
    self.wv = tf.keras.layers.Dense(d_model,activation='relu',use_bias='true')

    self.dense = tf.keras.layers.Dense(d_model)#,activation='relu',use_bias='true')

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask=None):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
#     q=PReLU()(q)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
#     k=PReLU()(k)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
#     v=PReLU()(v)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output#, attention_weights



class CrossStitch(tf.keras.layers.Layer):

    """Cross-Stitch implementation according to arXiv:1604.03539
    Implementation adapted from https://github.com/helloyide/Cross-stitch-Networks-for-Multi-task-Learning"""

    def __init__(self, num_tasks, *args, **kwargs):
        """initialize class variables"""
        self.num_tasks = num_tasks
        super(CrossStitch, self).__init__(**kwargs)

    def build(self, input_shape):
        """initialize the kernel and set the instance to 'built'"""
        self.kernel = self.add_weight(name="kernel",
                                      shape=(self.num_tasks,
                                             self.num_tasks),
                                      initializer='identity',
                                      trainable=True)
        super(CrossStitch, self).build(input_shape)

    def call(self, xl):
        """
        called by TensorFlow when the model gets build. 
        Returns a stacked tensor with num_tasks channels in the 0 dimension, 
        which need to be unstacked.
        """
        if (len(xl) != self.num_tasks):
            # should not happen
            raise ValueError()

        out_values = []
        for this_task in range(self.num_tasks):
            this_weight = self.kernel[this_task, this_task]
            out = tf.math.scalar_mul(this_weight, xl[this_task])
            for other_task in range(self.num_tasks):
                if this_task == other_task:
                    continue  # already weighted!
                other_weight = self.kernel[this_task, other_task]
#                 out += tf.math.scalar_mul(other_weight, xl[other_task])
            out_values.append(out)
        # HACK!
        # unless we stack, and then unstack the tensors, TF (2.0.0) can't follow
        # the graph, so it aborts during model initialization.
        # return tf.stack(out_values, axis=0)
        return out_values[0],out_values[1]

    def compute_output_shape(self, input_shape):
        return [self.num_tasks] + input_shape

    def get_config(self):
        """implemented so keras can save the model to json/yml"""
        config = {
            "num_tasks": self.num_tasks
        }
        base_config = super(CrossStitch, self).get_config()
        return dict(list(config.items()) + list(base_config.items()))

    
 


import keras.backend as K
from keras.optimizers import Adam

class AdamW(Adam):
    def __init__(self, learning_rate=0.001, weight_decay=0.025, **kwargs):
        super(AdamW, self).__init__(learning_rate, **kwargs)
        self.weight_decay = K.variable(weight_decay, name='weight_decay')

    def get_updates(self, loss, params):
        # Apply weight decay to the parameters
        decay_updates = [
            K.update_add(param, -self.weight_decay * param)
            for param in params
            if param.name.endswith('kernel:0')  # Apply weight decay only to kernel weights
        ]

        # Call the parent get_updates() method to get the remaining updates
        updates = super(AdamW, self).get_updates(loss, params)

        # Combine the weight decay updates and other updates
        updates.extend(decay_updates)

        return updates

    
import keras.backend as K
from keras.optimizers import Optimizer

class AMSGrad(Optimizer):
    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7, **kwargs):
        super(AMSGrad, self).__init__(**kwargs)
        self.learning_rate = learning_rate
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.epsilon = epsilon
        self.iterations = K.variable(0)
        self.m = None
        self.v = None

def get_updates(self, loss, params):
    grads = self.get_gradients(loss, params)
    self.updates = [K.update_add(self.iterations, 1)]

    t = self.iterations + 1
    lr_t = self.learning_rate * (K.sqrt(1.0 - K.pow(self.beta_2, t)) / (1.0 - K.pow(self.beta_1, t)))

    shapes = [K.int_shape(p) for p in params]
    self.m = [K.zeros(shape) for shape in shapes]
    self.v = [K.zeros(shape) for shape in shapes]

    for p, g, m, v in zip(params, grads, self.m, self.v):
        m_t = (self.beta_1 * m) + (1.0 - self.beta_1) * g
        v_t = (self.beta_2 * v) + (1.0 - self.beta_2) * K.square(g)
        v_hat = K.maximum(v, v_t)  # AMSGrad modification

        p_t = p - lr_t * m_t / (K.sqrt(v_hat) + self.epsilon)

        self.updates.append(K.update(m, m_t))
        self.updates.append(K.update(v, v_t))
        self.updates.append(K.update(p, p_t))

    return self.updates


In [5]:

def generate_network_att1(x_in1,x_in2,cell, inDrop, drop,ddi,graph1,graph2):
    # fill the architecture params from dict


    cell_layers = [512,265,128]

    snp_layers = [512,128]

    ddi_layers=[1024,512,256]
    layers=[256,512,256]
#     g_layers=[258,128]
    g_layers=[327,128]
    dsn1_layers = [1024,512,256]
    dsn2_layers = [1024,512,256]
   
    l2_reg = 1e-3  # L2 regularization rate

    

    snp_layers = [2048,1024,2048]
    # contruct two parallel networks
    for l in range(len(dsn1_layers)):
        if l == 0:
            x_in1    = Input(shape=(x_in1.shape[1],),name="input1")
            middle_layer = Dense(int(dsn1_layers[l]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(x_in1)
            middle_layer = Dropout(float(inDrop))(middle_layer) 
        elif l == (len(dsn1_layers)-1):
            dsn1_output = Dense(int(dsn1_layers[l]), activation='linear',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(middle_layer)
        else:
            middle_layer = Dense(int(dsn1_layers[l]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(middle_layer)
            middle_layer = Dropout(float(drop))(middle_layer)

    
    concatModel1 =  dsn1_output

    
    for l in range(len(dsn2_layers)):
        if l == 0:
            x_in2    = Input(shape=(x_in2.shape[1],),name="input2")
            middle_layer = Dense(int(dsn2_layers[l]), activation='relu', use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(x_in2)
            middle_layer = Dropout(float(inDrop))(middle_layer)
        elif l == (len(dsn2_layers)-1):
            dsn2_output = Dense(int(dsn2_layers[l]), activation='linear',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(middle_layer)
        else:
            middle_layer = Dense(int(dsn2_layers[l]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(middle_layer)
            middle_layer = Dropout(float(drop))(middle_layer)
    
    concatModel2 = dsn2_output
    

    for cell_layer in range(len(cell_layers)):
      if cell_layer == 0:
        input_cell    = Input(shape=(cell[0].shape[0],))  #

        cellFC = Dense(int(cell_layers[cell_layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(input_cell)
        cellFC = Dropout(float(drop))(cellFC)

      elif cell_layer == (len(cell_layers)-1):
        cellFC = Dense(int(cell_layers[cell_layer]), activation='linear',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(cellFC)

      else:
        cellFC = Dense(int(cell_layers[cell_layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(cellFC)
        cellFC = Dropout(float(drop))(cellFC)
#        


    graph_in1=Input(shape=(graph1.shape[1],),name='graph1')

    for g_layer in range(len(g_layers)):
      if g_layer == 0:

        graph_out1 = Dense(int(g_layers[g_layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(graph_in1)
        graph_out1 = Dropout(float(drop))(graph_out1)
#         
        
      elif g_layer == (len(g_layers)-1):
        graph_out1 = Dense(int(g_layers[g_layer]), activation='linear',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(graph_out1)
#         
      else:
        graph_out1 = Dense(int(g_layers[g_layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(graph_out1)
        graph_out1 = Dropout(float(drop))(graph_out1)

    
    
    graph_in2=Input(shape=(graph2.shape[1],),name='graph2')

    for g_layer in range(len(g_layers)):
      if g_layer == 0:

        graph_out2 = Dense(int(g_layers[g_layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(graph_in2)
        graph_out2 = Dropout(float(drop))(graph_out2)
        
      elif g_layer == (len(g_layers)-1):
        graph_out2 = Dense(int(g_layers[g_layer]), activation='linear',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(graph_out2)


      else:
        graph_out2 = Dense(int(g_layers[g_layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(graph_out2)
        graph_out2 = Dropout(float(drop))(graph_out2)
    
    concatModel1=concatenate([concatModel1,graph_out1])
    concatModel2=concatenate([concatModel2,graph_out2])
    
    
    for layer in range(len(layers)):
      if layer == 0:

        concatModel1 = Dense(int(layers[layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(concatModel1)
        concatModel1 = Dropout(float(drop))(concatModel1)

        
      elif layer == (len(layers)-1):
        concatModel1 = Dense(int(layers[layer]), activation='linear',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(concatModel1)

      else:
        concatModel1 = Dense(int(layers[layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(concatModel1)
        concatModel1 = Dropout(float(drop))(concatModel1)
    
    
    for layer in range(len(layers)):
      if layer == 0:

        concatModel2 = Dense(int(layers[layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(concatModel2)
        concatModel2 = Dropout(float(drop))(concatModel2)
        
      elif layer == (len(layers)-1):
        concatModel2 = Dense(int(layers[layer]), activation='linear',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(concatModel2)

      else:
        concatModel2 = Dense(int(layers[layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(concatModel2)
        concatModel2 = Dropout(float(drop))(concatModel2)
    
    
    ddi_out1=Input(shape=(ddi.shape[1],),name='ddi')

    for ddi_layer in range(len(ddi_layers)):
      if ddi_layer == 0:

        ddi_out = Dense(int(ddi_layers[ddi_layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(ddi_out1)
        ddi_out = Dropout(float(drop))(ddi_out)

        
      elif ddi_layer == (len(ddi_layers)-1):
        ddi_out = Dense(int(ddi_layers[ddi_layer]), activation='linear',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(ddi_out)

      else:
        ddi_out = Dense(int(ddi_layers[ddi_layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(ddi_out)
        ddi_out = Dropout(float(drop))(ddi_out)

    
    mlayer =MultiHeadAttention(d_model=cellFC.shape[1], num_heads=4)
    cellFC1= mlayer(cellFC,ddi_out,ddi_out)
    cellFC1 = Reshape([cellFC1.shape[2]])(cellFC1)
    cellFC=concatenate([cellFC1,cellFC])

    
    concatModel=concatenate([concatModel2,concatModel1,cellFC])
    concatModel=BatchNormalization()(concatModel)
    
        
   
#    
    layer1 =MultiHeadAttention(d_model=concatModel.shape[1], num_heads=4)
    a_task1= layer1(concatModel,concatModel,concatModel)
    layer2 = MultiHeadAttention(d_model=concatModel.shape[1], num_heads=4)
    a_task2= layer2(concatModel,concatModel,concatModel)
    task11 = Reshape([a_task1.shape[2]])(a_task1)
    task22 = Reshape([a_task2.shape[2]])(a_task2)
    task1=concatenate([task11,concatModel])
    task2=concatenate([task22,concatModel])

    r_task1,r_task2 = CrossStitch(2)([task1,task2])

    
    r_task1=Dense(2048,activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(r_task1)
    
    r_task2=Dense(2048, activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(r_task2)

                             
    r_task1,r_task2 = CrossStitch(2)([r_task1,r_task2])

    
    r_task1=concatenate([r_task1,task1])
    r_task2=concatenate([r_task2,task2])

    
    
    r_task1 = Dense(128, activation='linear',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001),name='fsynergy1')(r_task1)
    r_task1=PReLU()(r_task1)
    r_task2 = Dense(128, activation='relu',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001),name='fclass1')(r_task2)

   
    r_task1 = Dense(64, activation='linear',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001),name='fsynergy2')(r_task1)
    r_task1=PReLU()(r_task1)
    r_task2 = Dense(64, activation='relu',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001),name='fclass2')(r_task2)
#   

    
    snp_output1 = Dense(1, activation='linear',name='synergy')(r_task1)
    snp_output2 = Dense(3, activation='sigmoid',name='class')(r_task2)
    

    model = Model(inputs=[x_in1,x_in2,input_cell,ddi_out1,graph_in1,graph_in2],outputs= [snp_output1,snp_output2])

    print(model.summary())
    return model


def trainer_att1(model, l_rate, train_f_drug1,train_f_drug2,train_cell_line,train_synergy,train_class,train_ddi,train_graph1,train_grpah2, epo, batch_size, earlyStop):

    optimizer = AdamW(learning_rate=0.00001, weight_decay=0.025)

    model.compile(optimizer=optimizer,loss={'synergy':'mse','class':'categorical_crossentropy'})

    model.fit([train_f_drug1,train_f_drug2,train_cell_line,train_ddi,train_graph1,train_graph2],[train_synergy,train_class],shuffle=True, epochs=epo, batch_size=batch_size,verbose=1) 
                  

    return model


In [6]:
#n

l_rate = 0.00001
inDrop = 0.2
drop = 0.2   
max_epoch =1000
batch_size = 64 
earlyStop_patience = 20



model_att1= generate_network_att1(train_f_drug1,train_f_drug2,train_cell_line, inDrop, drop,train_ddi,train_graph1,train_graph2)

train_class1 = keras.utils.to_categorical(train_class, num_classes=3)
model_att1=trainer_att1(model_att1, l_rate, train_f_drug1,train_f_drug2,train_cell_line,train_synergy,train_class1,train_ddi,train_graph1,train_graph2,max_epoch, batch_size,
                                earlyStop_patience)

ap111,ap221= model_att1.predict( [test_f_drug1,test_f_drug2,test_cell_line,test_ddi,test_graph1,test_graph2])

positive_negative_indices = np.where(test_class != 2)
ap221=ap221[positive_negative_indices]
test_class12=np.array(test_class)
test_class12=test_class12[positive_negative_indices]
ap221=ap221[:,1]


ap11=[]
test_synergy1=[]
l=len(ap111)
l1=int(l/2)
for i in range(l1) :
    ap=(ap111[i]+ap111[i+l1])/2
    ap11.append(ap)
    aap=(test_synergy[i]+test_synergy[i+l1])/2
    test_synergy1.append(aap)
    
    
ap22=[]
test_class1=[]
l=len(ap221)
l1=int(l/2)
for i in range(l1) :
    ap=(ap221[i]+ap221[i+l1])/2
    ap22.append(ap)
    aap=(test_class12[i]+test_class12[i+l1])/2
    test_class1.append(aap)
    
    
    
asynergy_error1=mean_squared_error(test_synergy1, ap11)
asynergy_error11=mean_absolute_error(test_synergy1, ap11)
asynergy_error21=r2_score(test_synergy1, ap11)

print("msynergy_mean_squared_error",asynergy_error1)
print("mclass_mean_squared_error",asynergy_error11)
print("msynergy_r2_score",asynergy_error21)

asynergy_pear1= pearsonr(test_synergy1, ap11)
asynergy_spear1= spearmanr(test_synergy1, ap11)
print("msynergy_pear",asynergy_pear1)
print("msynergy_spear",asynergy_spear1)

aap2=convert_tobin(ap22)

aclass_error1=roc_auc_score(test_class1, ap22)
aclass_error11=accuracy_score(test_class1, aap2)
aclass_error21=cohen_kappa_score(test_class1, aap2)

print("msclass_roc_curve",aclass_error1)
print("mclass_accuracy_scorer",aclass_error11)
print("mclass_cohen_kappa_score",aclass_error21)


aclass_pear1= precision_score(test_class1, aap2)
aclass_spear1= average_precision_score(test_class1, ap22)
print("mclass_precision_score",aclass_pear1)
print("mclass_average_precision_score",aclass_spear1)


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input2 (InputLayer)            [(None, 394)]        0           []                               
                                                                                                  
 input1 (InputLayer)            [(None, 394)]        0           []                               
                                                                                                  
 dense_3 (Dense)                (None, 1024)         404480      ['input2[0][0]']                 
                                                                                                  
 dense (Dense)                  (None, 1024)         404480      ['input1[0][0]']                 
                                                                                            

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



570/570 [==============================] - 11s 19ms/step - loss: 444.4756 - synergy_loss: 159.5120 - class_loss: 0.5660
Epoch 108/1000
570/570 [==============================] - 11s 19ms/step - loss: 444.0124 - synergy_loss: 159.1765 - class_loss: 0.5626
Epoch 109/1000
570/570 [==============================] - 11s 20ms/step - loss: 444.4140 - synergy_loss: 159.7099 - class_loss: 0.5629
Epoch 110/1000
570/570 [==============================] - 11s 19ms/step - loss: 443.3781 - synergy_loss: 158.8124 - class_loss: 0.5587
Epoch 111/1000
570/570 [==============================] - 11s 19ms/step - loss: 442.9231 - synergy_loss: 158.5319 - class_loss: 0.5617
Epoch 112/1000
570/570 [==============================] - 12s 20ms/step - loss: 439.7448 - synergy_loss: 155.4871 - class_loss: 0.5606
Epoch 113/1000
570/570 [==============================] - 11s 19ms/step - loss: 438.0224 - synergy_loss: 153.9136 - class_loss: 0.5587
Epoch 114/1000
570/570 [==============================] - 11s 19ms/ste

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



570/570 [==============================] - 12s 20ms/step - loss: 403.6844 - synergy_loss: 123.1691 - class_loss: 0.5299
Epoch 139/1000
570/570 [==============================] - 11s 19ms/step - loss: 404.1815 - synergy_loss: 123.7895 - class_loss: 0.5303
Epoch 140/1000
570/570 [==============================] - 11s 19ms/step - loss: 403.0649 - synergy_loss: 122.7994 - class_loss: 0.5272
Epoch 141/1000
570/570 [==============================] - 12s 20ms/step - loss: 403.5770 - synergy_loss: 123.4661 - class_loss: 0.5274
Epoch 142/1000
570/570 [==============================] - 11s 19ms/step - loss: 401.6380 - synergy_loss: 121.6596 - class_loss: 0.5300
Epoch 143/1000
570/570 [==============================] - 11s 19ms/step - loss: 401.5667 - synergy_loss: 121.7705 - class_loss: 0.5264
Epoch 144/1000
570/570 [==============================] - 12s 21ms/step - loss: 398.6132 - synergy_loss: 118.9709 - class_loss: 0.5267
Epoch 145/1000
570/570 [==============================] - 11s 19ms/ste

In [11]:
   

from IPython.display import FileLink    
np.savetxt('npred_syn3.csv', ap11 ,delimiter=',')
FileLink(r'npred_syn3.csv')

np.savetxt('npred_cls3.csv', ap22 ,delimiter=',') 
FileLink(r'npred_cls3.csv')
 
np.savetxt('ntest_syn3.csv', test_synergy1 ,delimiter=',')
FileLink(r'ntest_syn3.csv')

np.savetxt('ntest_cls3.csv', test_class1 ,delimiter=',')
FileLink(r'ntest_cls3.csv')



/kaggle/working/ntest_cls3.csv

##### 